# Setup

In [10]:
%cd /kaggle/working/
%rm -rf mlproduction
!git clone https://github.com/daoanhkhoa123/mlproduction.git
%cd /kaggle/working/mlproduction
!ls

/kaggle/working
Cloning into 'mlproduction'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (282/282), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 282 (delta 163), reused 209 (delta 91), pack-reused 0 (from 0)
Receiving objects: 100% (282/282), 3.53 MiB | 5.60 MiB/s, done.
Resolving deltas: 100% (163/163), done.
/kaggle/working/mlproduction
main.py  pyproject.toml  README.md  runtime  scripts  sqls  src  uv.lock


In [11]:
!chmod +x scripts/*.sh
!scripts/cloud_setup.sh

Resolved 69 packages in 201ms                                        
# This file was autogenerated by uv via the following command:
#    uv pip compile pyproject.toml -o requirements.txt --extra-index-url https://pypi.org/simple --emit-index-url
--index-url https://pypi.org/simple
--extra-index-url https://pypi.org/simple

aiohappyeyeballs==2.6.1
    # via aiohttp
aiohttp==3.13.2
    # via fsspec
aiosignal==1.4.0
    # via aiohttp
annotated-types==0.7.0
    # via pydantic
attrs==25.4.0
    # via aiohttp
certifi==2025.11.12
    # via requests
charset-normalizer==3.4.4
    # via requests
filelock==3.20.1
    # via
    #   huggingface-hub
    #   torch
    #   transformers
frozenlist==1.8.0
    # via
    #   aiohttp
    #   aiosignal
fsspec==2025.12.0
    # via
    #   huggingface-hub
    #   lightning
    #   pytorch-lightning
    #   torch
hf-xet==1.2.0
    # via huggingface-hub
huggingface-hub==0.36.0
    # via
    #   tokenizers
    #   transformers
idna==3.11
    # via
    #   reque

In [12]:
!cat requirements.txt

# This file was autogenerated by uv via the following command:
#    uv pip compile pyproject.toml -o requirements.txt --extra-index-url https://pypi.org/simple --emit-index-url
--index-url https://pypi.org/simple
--extra-index-url https://pypi.org/simple

aiohappyeyeballs==2.6.1
    # via aiohttp
aiohttp==3.13.2
    # via fsspec
aiosignal==1.4.0
    # via aiohttp
annotated-types==0.7.0
    # via pydantic
attrs==25.4.0
    # via aiohttp
certifi==2025.11.12
    # via requests
charset-normalizer==3.4.4
    # via requests
filelock==3.20.1
    # via
    #   huggingface-hub
    #   torch
    #   transformers
frozenlist==1.8.0
    # via
    #   aiohttp
    #   aiosignal
fsspec==2025.12.0
    # via
    #   huggingface-hub
    #   lightning
    #   pytorch-lightning
    #   torch
hf-xet==1.2.0
    # via huggingface-hub
huggingface-hub==0.36.0
    # via
    #   tokenizers
    #   transformers
idna==3.11
    # via
    #   requests
    #   yarl
isort==7.0.0
    # via mlproduction (pyproject.toml)


# Run

In [13]:
import torch

In [14]:
!mlflow

/bin/bash: line 1: mlflow: command not found


In [4]:
import pandas as pd
df = pd.read_csv("data/data.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/data.csv'

NameError: name 'tokenizer' is not defined

In [13]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModelForMaskedLM.from_pretrained("vinai/phobert-base")

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [34]:
import torch

text = "(attended): KhoaDACE190399 had attended this activity / Đào Anh Khoa đã tham gia hoạt động này (absent): KhoaDACE190399 had NOT attended this activity / Đào Anh Khoa đã vắng mặt buổi này (-): no data was given / chưa có dữ liệu"

tokens = tokenizer(text, return_tensors="pt")
print(tokens)

with torch.no_grad():
    output= model(**tokens)

output["logits"].shape

{'input_ids': tensor([[    0, 14157,  3148,  1187,  6374,  3995, 37272,    27,  8044,  1850,
         16311,  9594,  4917, 12627,  4424, 30498,  3148,  7104, 39573, 22304,
          3065, 20712,  8233,    95,  5483,   157,  3720,    14,  7520,  3931,
         11297,  2697,    23, 14157,  3377,  1302,  6502, 37272,    27,  8044,
          1850, 16311,  9594,  4917, 12627,  4424, 30498, 13986,  1085,  3148,
          7104, 39573, 22304,  3065, 20712,  8233,    95,  5483,   157,  3720,
            14,  3543,   373,   391,    23, 14157,   770, 37272,    27,  4397,
         16819, 26431,  2667,  1946,    95,   102,    10,  7015,  1934,     2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 

torch.Size([1, 80, 64001])